In [ ]:
import os
import asyncio
import aiohttp
import json
import re
from random import shuffle

In [ ]:
import dotenv
dotenv.load_dotenv()

import nest_asyncio
nest_asyncio.apply()

True

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
llm_gpt3 = AzureChatOpenAI(
    model_name="gpt-35-turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

llm_gpt4o = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

llm_gpt5 = AzureChatOpenAI(
    model_name="gpt-5",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Test

In [ ]:
with open("users.json", "r", encoding="utf-8") as f:
    data = json.load(f)

user_data = []
for user in data["users"]:
    user_id = user["user_id"]
    available = user["available_cards"]
    selected = user["selected_cards"]

    user_data.append((user_id, available, selected))



In [ ]:
with open("human_prompt.txt", "r", encoding="utf-8") as f:
    human_prompt = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt = human_prompt.replace("\n", " ").replace('"', '\\"')
    human_prompt = human_prompt.replace("$", "{").replace("%", "}")

with open("system_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

In [ ]:
from pathlib import Path
import re

def parse_json_safe(user_id, llm_response, deleted):
    results_path = Path("results.json")
    try:
        s = (llm_response or "").strip()

        m = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", s, flags=re.IGNORECASE)
        if m:
            candidate = m.group(1).strip()
        else:
            candidate = s

        first_brace = candidate.find("{")
        first_bracket = candidate.find("[")
        starts = [i for i in (first_brace, first_bracket) if i != -1]
        if starts:
            start = min(starts)
            end = max(candidate.rfind("}"), candidate.rfind("]"))
            if end != -1 and end >= start:
                candidate = candidate[start:end+1]

        parsed = json.loads(candidate)

        if isinstance(parsed, dict):
            parsed["user_id"] = user_id
            parsed["deleted"] = deleted
        else:
            parsed = {"user_id": user_id, "data": parsed}

        if results_path.exists():
            with open(results_path, "r", encoding="utf-8") as f:
                try:
                    results = json.load(f)
                    if not isinstance(results, list):
                        results = [results]
                except json.JSONDecodeError:
                    results = []
        else:
            results = []

        results.append(parsed)

        with open(results_path, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print("Resultado agregado correctamente a results.json")

    except json.JSONDecodeError as e:
        print(f"X Error al parsear el JSON: {e}")
    except Exception as e:
        print(f"! Error inesperado: {e}")

In [ ]:
for user in user_data:
    print(f"Recomendando para el usuario {user_id}...")
    print("-" * 50)

    user_id, available, selected = user
    print(f"Cartas disponibles: {len(available)}, Cartas seleccionadas: {len(selected)}")

    shuffle(available)
    shuffle(selected)

    to_delete = selected[-3:]
    selected_copy = selected[:-3]

    print(f"Cartas seleccionadas (para prompt): {len(selected_copy)}")

    rendered_human_prompt = human_prompt.format(
        CARTAS_DISPONIBLES=json.dumps(available),
        CARTAS_SELECCIONADAS=json.dumps(selected_copy)
    )

    messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=rendered_human_prompt),
    ]
    ai_msg = llm_gpt4o.invoke(messages).content

    parse_json_safe(user_id, ai_msg, to_delete)

Recomendando para el usuario 50...
--------------------------------------------------
Cartas disponibles: 121, Cartas seleccionadas: 5
Eliminando cartas seleccionadas: ['Minions', 'Golem', 'Lumberjack']
Cartas seleccionadas (para prompt): 2
✅ Resultado agregado correctamente a results.json
Recomendando para el usuario 50...
--------------------------------------------------
Cartas disponibles: 121, Cartas seleccionadas: 8
Eliminando cartas seleccionadas: ['Firecracker', 'Poison', 'Inferno Tower']
Cartas seleccionadas (para prompt): 5
✅ Resultado agregado correctamente a results.json
Recomendando para el usuario 60...
--------------------------------------------------
Cartas disponibles: 121, Cartas seleccionadas: 8
Eliminando cartas seleccionadas: ['Miner', 'Wall Breakers', 'Mega Knight']
Cartas seleccionadas (para prompt): 5
✅ Resultado agregado correctamente a results.json
Recomendando para el usuario 70...
--------------------------------------------------
Cartas disponibles: 121, C

In [31]:
# from openai import AzureOpenAI
# import dotenv

# client = AzureOpenAI(
#     azure_endpoint="https://victo-mhcmsfx4-eastus2.cognitiveservices.azure.com/",
#     api_key=dotenv.get_key(dotenv.find_dotenv(), "AZURE_OPENAI_API_KEY"),
#     api_version="2024-05-01-preview"
# )

# response = client.chat.completions.create(
#     model="grok-4-fast-reasoning",
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": rendered_human_prompt}
#     ],
#     temperature=0
# )

# print(response.choices[0].message.content)